In [44]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import torch.optim as optim
import torch.nn.init

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [45]:
cifar10_train=datasets.CIFAR10(root="../data00/cifar10_data",
                               train=True,
                               download=True,
                               transform=transforms.Compose([transforms.ToTensor()]))
cifar10_test=datasets.CIFAR10(root="../data00/cifar10_data",
                               train=False,
                               download=True,
                               transform=transforms.Compose([transforms.ToTensor()]))

device setting

In [46]:
device='cuda' if torch.cuda.is_available() else 'cpu' #텐서 만드는거
torch.manual_seed(0)
if device =='cuda':
  torch.cuda.manual_seed_all(0)

data setting

In [47]:
from copy import deepcopy

learing_rate=0.001
epochs=10
batch_size=100
lowest_loss=np.inf
lowest_epoch=np.inf
early_stop = 5

In [48]:
train_loader = DataLoader(dataset=cifar10_train, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          drop_last=True)

test_loader = DataLoader(dataset=cifar10_test, 
                          batch_size=batch_size, 
                          shuffle=True, 
                          drop_last=True)

train_loader에서 X, Y를 각각 꺼내

In [49]:
for X, Y in train_loader:
  print(X.size(), Y.size())
  break #모양볼 목적이니

torch.Size([100, 3, 32, 32]) torch.Size([100])


CNN 구축

In [50]:
class CNN(nn.Module):
  def __init__(self): #모델정의
    super().__init__()

    self.layer1 = nn.Sequential(
      nn.Conv2d(3, 32, kernel_size=3, padding=1), #컨볼루션 32개 특징 탐지
      nn.ReLU(), 
      nn.MaxPool2d(kernel_size=2, stride=2)
    )

    #image shape(100, 3, 32, 32) 이미지 100, 채널 1, 크기 28*28
    # conv1 -> (3, 32, 32, 32) 채널값, 출력값, size
    # pool -> (3, 32, 16, 16) 값이 요렇게 된다.

    self.layer2 = nn.Sequential(
      nn.Conv2d(32, 64, kernel_size=3, padding=1), # 2의 지수승이 출력 64
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    #image shape(?, 32, 14, 14) 이미지수? 채널32, 크기 14*14
    # conv1 -> (32, 64, 14, 14) 채널값, 출력값, size
    # pool -> (32, 64, 7, 7) 값이 요렇게 된다.

    self.layer3 = nn.Sequential(
      nn.Conv2d(64, 128, kernel_size=3, padding=1), # 2의 지수승이 출력 64
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )


    self.fc=nn.Sequential(
      nn.Linear(4*4*128, 100, bias=True),
      nn.LeakyReLU(), 
      nn.Linear(100, 10, bias=True)
    )
    #self.fc=nn.Linear(4*4*128, 100, bias=True)
    # nn.init.xavier_uniform_(self.fc.weight)

    #모델 실행

  def forward(self, x):
    out=self.layer1(x) #100개의 self 값이 들어옴
    out=self.layer2(out)
    out=self.layer3(out)    
    out=out.view(out.size(0),-1)  #이미지 1차원으로 펼침
    out=self.fc(out)
    return out    




In [51]:

model = CNN().to(device) #device에서 CNN을 함
crit=nn.CrossEntropyLoss().to(device) #손실함수 계산
optimizer=optim.Adam(model.parameters(), lr=learing_rate) #초기는 lr, 이후는 Adam

모델출력하면 아래와 같이 나옴

In [52]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=2048, out_features=100, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)

total 배치 구하기

In [53]:
total_batch=len(train_loader)
total_batch
test_batch = len(test_loader)

In [54]:
for epoch in range(epochs):
  avg_cost=0

  #배치를 돌려야하니 for
  for X, Y in train_loader:
    X=X.to(device)
    Y=Y.to(device)

    optimizer.zero_grad()
    y_hat=model(X) #결과
    cost=crit(y_hat, Y)
    cost.backward()
    optimizer.step()

    avg_cost+=cost
  avg_cost=avg_cost/total_batch #햇갈릴 것 같으면 이렇게 쓸라심. 현재 코스트 평균코스트
  print(f'epoch:{epoch}, cost:{avg_cost}')

epoch:0, cost:1.5564669370651245
epoch:1, cost:1.15193510055542
epoch:2, cost:0.973167896270752
epoch:3, cost:0.8463259339332581
epoch:4, cost:0.7535978555679321
epoch:5, cost:0.6764012575149536
epoch:6, cost:0.6097567081451416
epoch:7, cost:0.5525524616241455
epoch:8, cost:0.4896961748600006
epoch:9, cost:0.43710511922836304


In [55]:
test_X = torch.FloatTensor(cifar10_test.data).permute(0,3,1,2)
print(test_X.size())
test_Y = torch.LongTensor(cifar10_test.targets)
print(test_Y.size())

torch.Size([10000, 3, 32, 32])
torch.Size([10000])


In [56]:
with torch.no_grad(): 
  x_test=test_X.view(test_X.size(0), 3, 32, 32).float().to(device)
  y_test=test_Y.to(device)

  pred=model(x_test)
  correct_pred=torch.argmax(pred, -1) == y_test 
  accuracy=correct_pred.float().mean() 
  # accuracy=correct_pred.sum()/len(pred)
  print(f'Accuracy:{accuracy}') #정확도 몇퍼센트인지

Accuracy:0.5467000007629395


In [57]:
with torch.no_grad():

  x_test=test_X.to(device)
  y_test=test_Y.to(device)

  pred=model(x_test)
  correct_pred=torch.argmax(pred, -1) == y_test 
  accuracy=correct_pred.float().mean() 
  # accuracy=correct_pred.sum()/len(pred)
  print(f'Accuracy:{accuracy}') #정확도 몇퍼센트인지

Accuracy:0.5467000007629395


단을 최대한 쌓은 거야.

단을 적게 쌓으면 더 잘 나올수도 있어.


In [58]:
with torch.no_grad():
  avg_accuracy = 0

  for x_test,y_test in test_loader:
    x_test=test_X.to(device)
    y_test=test_Y.to(device)

    pred=model(x_test)
    correct_pred=torch.argmax(pred, -1) == y_test 
    accuracy=correct_pred.float().mean() 
    # accuracy=correct_pred.sum()/len(pred)
  print(f'Accuracy:{accuracy}') #정확도 몇퍼센트인지

Accuracy:0.5467000007629395
